In [3]:
import ndjson
import cv2
import os
import pandas as pd
import numpy as np

# Open and read the NDJSON file
with open('D:\\MANAGER-ANGRY-METER\\video-input\\videos.ndjson', 'r') as f:
    data = ndjson.load(f)


print(f"Combined records: {len(data)}")


Combined records: 69


In [4]:
df_meta = pd.DataFrame(data)
df_meta.drop(columns=['media_attributes'], inplace=True)
df_meta['data_row'] = df_meta['data_row'].astype(str).str[8:37]
df_meta.rename(columns={'data_row': 'id'}, inplace=True)
df_meta.rename(columns={'metadata_fields': 'emotion'}, inplace=True)
df_meta.head()

,id,emotion
0,cmhpke9p4cjbs0782sob057xx.mp4,[{'value': 'Surprised'}]
1,cmhpke9p4cjbt0782363n2x0u.mp4,[{'value': 'Surprised'}]
2,cmhpke9p4cjbu0782assgc5x9.mp4,[{'value': 'Surprised'}]
3,cmhpke9p4cjbv0782ah5bfrg4.mp4,[{'value': 'Surprised'}]
4,cmhpke9p4cjbw0782wkp7yx3s.mp4,[{'value': 'Surprised'}]


In [4]:
# --- ROBUST DATA CLEANING ---
def clean_emotion_col(x):
    # 1. If it's a list (e.g., ['angry']), grab the first item
    if isinstance(x, list):
        if len(x) > 0:
            x = x[0]
        else:
            return "unknown"

    # 2. If it's a dictionary (e.g., {'label': 'angry'}), grab the value
    if isinstance(x, dict):
        # Try common keys, or just grab the first value found
        x = x.get('label', x.get('emotion', list(x.values())[0]))

    # 3. Convert whatever is left to a string and lower-case it
    text = str(x).lower()
    
    # 4. Remove any leftover garbage characters
    text = text.replace("'", "").replace('"', "").strip()
    return text

# Apply the fix
print("Cleaning data...")
df_meta['emotion'] = df_meta['emotion'].apply(clean_emotion_col)
# --- VERIFY ---
print("\nUnique emotions found:")
print(df_meta['emotion'].unique())

Cleaning data...

Unique emotions found:
['surprised' 'sad' 'neutral' 'happy' 'disgust' 'angry']


In [8]:
import cv2
import os
import pandas as pd
import numpy as np

# --- CONFIGURATION ---
video_folder = "D:\\machine project\\data\\videos"
output_folder = "data/processed_frames_HD" 
img_size = (128, 128) 

# --- FIX IS HERE ---
# 1. Define the path
cascade_path = r"C:\Users\PC\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml"

# 2. Check if file exists (Crucial because Windows paths can be tricky)
if not os.path.exists(cascade_path):
    print(f"❌ ERROR: Cannot find the Cascade XML at: {cascade_path}")
    print("   Please check the path again.")
    exit()

# 3. Actually LOAD the classifier object
face_cascade = cv2.CascadeClassifier(cascade_path)

# Verify it loaded correctly
if face_cascade.empty():
    print("❌ ERROR: Cascade file found but failed to load. Is it corrupt?")
    exit()
else:
    print(f"✅ Face Detector loaded successfully from: {cascade_path}")


# --- START HARVESTING ---
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"🚜 Starting HD Harvest from {len(df_meta)} videos...")

count = 0
faces_found_count = 0

for index, row in df_meta.iterrows():
    video_filename = row['id']
    emotion_label = row['emotion']
    
    # Check extension
    if not video_filename.endswith('.mp4'):
        video_filename += ".mp4"
        
    video_path = os.path.join(video_folder, video_filename)
    
    # Skip if video file missing
    if not os.path.exists(video_path):
        continue

    # Create Emotion Folder
    label_path = os.path.join(output_folder, emotion_label)
    os.makedirs(label_path, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 1. Convert to Gray (Face detection needs grayscale)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 2. Detect Faces
        # scaleFactor=1.1 -> Scans image at different sizes (10% increments)
        # minNeighbors=4  -> Higher = fewer false positives, but might miss faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
        
        # 3. If a face is found, crop it
        if len(faces) > 0:
            for (x, y, w, h) in faces:
                # Crop: [y_start:y_end, x_start:x_end]
                face_img = frame[y:y+h, x:x+w]
                
                try:
                    # High Quality Resize
                    face_resized = cv2.resize(face_img, img_size, interpolation=cv2.INTER_AREA)
                    
                    # Save
                    clean_name = video_filename.replace('.mp4', '')
                    save_name = f"{clean_name}_f{frame_id}.jpg"
                    cv2.imwrite(os.path.join(label_path, save_name), face_resized)
                    
                    faces_found_count += 1
                except Exception as e:
                    pass # Skip if resize crashes (rare)
                
                # Only take the first face to avoid duplicates/crowd
                break 
        
        frame_id += 1
        count += 1

    cap.release()

print(f"✅ HD Harvest Complete!")
print(f"   Scanned {count} total frames.")
print(f"   Saved {faces_found_count} high-quality face images to '{output_folder}'")

✅ Face Detector loaded successfully from: C:\Users\PC\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml
🚜 Starting HD Harvest from 69 videos...
✅ HD Harvest Complete!
   Scanned 4677 total frames.
   Saved 4669 high-quality face images to 'data/processed_frames_HD'


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- CONFIGURATION ---
# We use the HD folder for BOTH training and validation
dataset_path = "data/processed_frames_HD" 
img_height, img_width = 128, 128  # Match the size you harvested!

# 1. LOAD DATA
datagen = ImageDataGenerator(
    rescale=1.0/255,      # Normalize
    validation_split=0.2  # 20% split
)

print(f"📂 Loading Training Data from {dataset_path}...")
train_ds = datagen.flow_from_directory(
    dataset_path,             # FIXED PATH
    target_size=(img_height, img_width), # FIXED SIZE (128x128)
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

print(f"📂 Loading Validation Data from {dataset_path}...")
val_ds = datagen.flow_from_directory(
    dataset_path,             # FIXED PATH
    target_size=(img_height, img_width), # FIXED SIZE (128x128)
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 2. DEFINE MODEL (Updated for 128x128 input)
model = Sequential([
    # --- CNN PART ---
    # Input Shape is now 128x128x3
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Optional: Add a 4th layer since 128x128 is deeper
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    
    # --- FCN PART ---
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    # Output
    Dense(train_ds.num_classes, activation='softmax')
])

# 3. TRAIN
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 Starting Training on HD Data...")
history = model.fit(train_ds, epochs=15, validation_data=val_ds)

📂 Loading Training Data from data/processed_frames_HD...
Found 3622 images belonging to 6 classes.
📂 Loading Validation Data from data/processed_frames_HD...
Found 903 images belonging to 6 classes.
🚀 Starting Training on HD Data...


c:\Users\PC\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 27s 227ms/step - accuracy: 0.4412 - loss: 1.4205 - val_accuracy: 0.5194 - val_loss: 1.3153
Epoch 2/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - accuracy: 0.6676 - loss: 0.9010 - val_accuracy: 0.6002 - val_loss: 1.0317
Epoch 3/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - accuracy: 0.7852 - loss: 0.5919 - val_accuracy: 0.6600 - val_loss: 1.0751
Epoch 4/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - accuracy: 0.8490 - loss: 0.4146 - val_accuracy: 0.5703 - val_loss: 1.4805
Epoch 5/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.8876 - loss: 0.3203 - val_accuracy: 0.6412 - val_loss: 1.3061
Epoch 6/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.9194 - loss: 0.2351 - val_accuracy: 0.5327 - val_loss: 2.0926
Epoch 7/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - accuracy: 0.9224 - loss: 0.2223 - val_accuracy: 0.6711 - val_loss: 1.6405
Epoch 8/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.9268 - loss: 0.2050 -

In [5]:


# Open and read the NDJSON file
with open('D:\\MANAGER-ANGRY-METER\\video-input\\Audio.ndjson', 'r') as f:
    data_audio = ndjson.load(f)


print(f"Combined records: {len(data_audio)}")



Combined records: 50


In [6]:
df_meta_audio = pd.DataFrame(data)
df_meta_audio.drop(columns=['media_attributes'], inplace=True)
df_meta_audio['data_row'] = df_meta_audio['data_row'].astype(str).str[8:37]
df_meta_audio.rename(columns={'data_row': 'id'}, inplace=True)
df_meta_audio.rename(columns={'metadata_fields': 'emotion'}, inplace=True)
df_meta_audio.head()

,id,emotion
0,cmhpke9p4cjbs0782sob057xx.mp4,[{'value': 'Surprised'}]
1,cmhpke9p4cjbt0782363n2x0u.mp4,[{'value': 'Surprised'}]
2,cmhpke9p4cjbu0782assgc5x9.mp4,[{'value': 'Surprised'}]
3,cmhpke9p4cjbv0782ah5bfrg4.mp4,[{'value': 'Surprised'}]
4,cmhpke9p4cjbw0782wkp7yx3s.mp4,[{'value': 'Surprised'}]


In [8]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from transformers import Wav2Vec2Processor, TFWav2Vec2Model
import tensorflow as tf

# --- CONFIGURATION ---
video_folder = "D:\\Manager-angry-meter\\data\\audio"
MODEL_NAME = "facebook/wav2vec2-base-960h" # The Industry Standard for Speech
MAX_DURATION = 3 # seconds

# 1. LOAD PRETRAINED MODEL (The "Brain")
print("📥 Loading Pretrained Wav2Vec2 Model...")
processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
# We load the TensorFlow version of the model
# output_hidden_states=True lets us peek inside the "brain"
# Force it to use standard weights to avoid the iteration error
base_model = TFWav2Vec2Model.from_pretrained(MODEL_NAME, use_safetensors=False)
def extract_deep_features(path):
    try:
        # A. Load Audio (16kHz is mandatory for Wav2Vec2)
        audio, sr = librosa.load(path, sr=16000, duration=MAX_DURATION)
        
        # B. Pad/Truncate to ensure consistent length
        # 3 seconds * 16000 Hz = 48000 samples
        target_len = 16000 * MAX_DURATION
        if len(audio) < target_len:
            audio = np.pad(audio, (0, target_len - len(audio)))
        else:
            audio = audio[:target_len]

        # C. Prepare Input for the Model
        inputs = processor(audio, sampling_rate=16000, return_tensors="tf")
        
        # D. Run through Pretrained Model
        outputs = base_model(inputs.input_values)
        
        # E. Get the "Embeddings" (The last hidden state)
        # Shape: (1, Sequence_Length, 768) -> We average it to get (768,)
        hidden_states = outputs.last_hidden_state
        features = tf.reduce_mean(hidden_states, axis=1) 
        
        return features.numpy().flatten()
        
    except Exception as e:
        return None

# --- HARVEST LOOP ---
deep_features = []
labels = []

print(f"🧠 Extracting Deep Features from {len(df_meta_audio)} videos...")

for index, row in tqdm(df_meta.iterrows(), total=len(df_meta)):
    filename = row['id']
    if not filename.endswith('.mp4'):
        filename += ".mp4"
    path = os.path.join(video_folder, filename)
    
    if os.path.exists(path):
        feats = extract_deep_features(path)
        if feats is not None:
            deep_features.append(feats)
            labels.append(row['emotion'])

# Save to disk so you don't have to run this again
X_deep = np.array(deep_features)
print(f"✅ Extraction Complete! Feature Shape: {X_deep.shape}") 
# Shape should be (Num_Videos, 768) -> 768 is the size of Wav2Vec2 embeddings

📥 Loading Pretrained Wav2Vec2 Model...


tf_model.h5:   0%|          | 0.00/378M [00:00<?, ?B/s]


TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some layers from the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing TFWav2Vec2Model: ['lm_head', 'dropout_50']
- This IS expected if you are initializing TFWav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFWav2Vec2Model were initialized from the model checkpoint at facebook/wav2vec2-base-960h.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWav2Vec2Model for predictions with

🧠 Extracting Deep Features from 69 videos...


100%|██████████| 69/69 [00:00<00:00, 23030.95it/s]

✅ Extraction Complete! Feature Shape: (0,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 1. Prepare Labels
lb = LabelEncoder()
y_deep = to_categorical(lb.fit_transform(labels))

# 2. Split
X_train, X_test, y_train, y_test = train_test_split(X_deep, y_deep, test_size=0.2, random_state=42)

# 3. BUILD YOUR CUSTOM MODEL
# This is "Adding the 2 layers" on top of the pretrained knowledge
model_transfer = Sequential([
    # Input is 768 (The output of Wav2Vec2)
    tf.keras.Input(shape=(768,)),
    
    # --- YOUR ADDED LAYERS ---
    
    # Added Layer 1
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4), # High dropout because Wav2Vec2 features are rich
    
    # Added Layer 2
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    # Output Layer
    Dense(y_deep.shape[1], activation='softmax')
])

# 4. Train
model_transfer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 Training Transfer Learning Model...")
# This will be super fast because the "Heavy Lifting" (Wav2Vec2) is already done
history = model_transfer.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# 5. Save
model_transfer.save('wav2vec2_transfer_model.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- DEFINE MODEL ---
model_audio = Sequential()

# 1. Input Layer (Matches your extracted features)
model_audio.add(tf.keras.Input(shape=(X.shape[1],)))

# 2. First FCN Layer (Hidden Layer 1)
model_audio.add(Dense(256, activation='relu'))
model_audio.add(BatchNormalization()) # Helps training stability
model_audio.add(Dropout(0.3))         # Prevents overfitting

# 3. Second FCN Layer (Hidden Layer 2 - as requested)
model_audio.add(Dense(128, activation='relu'))
model_audio.add(BatchNormalization())
model_audio.add(Dropout(0.3))

# 4. Output Layer
model_audio.add(Dense(y.shape[1], activation='softmax'))

# --- COMPILE & TRAIN ---
model_audio.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 Starting Audio FCN Training...")
history_audio = model_audio.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test)
)